In [1]:
import pandas as pd
import numpy as np
from collections import Counter, defaultdict
from tqdm.notebook import tqdm
import requests
from requests.exceptions import ReadTimeout, ConnectTimeout
from math import radians, cos, sin, asin, sqrt
from geolib import geohash
import json
import re

In [2]:
# 导入数据
raw_addr = pd.read_csv('./raw_data/地址文本_20220728.txt')

c:\users\xx\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (3,28,29,51) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
raw_addr.shape

(904825, 103)

In [4]:
sample_addr = raw_addr.copy(deep=True)

## 查漏补缺

In [5]:
# # 百度
# def addr2code(address):
#     AK = 'qc3iiBPzq3tT0XvKSyuquSzDkmQreeqt'
#     url = 'http://api.map.baidu.com/geocoding/v3/?address={}&output=json&ak={}&callback=showLocation'.format(address,AK)
#     res = requests.get(url)
#     results = json.loads(re.findall(r'\((.*?)\)',res.text)[0])
#     return results['result']['location']

In [6]:
# 高德
def addr2code(geo):
    # parameters = {'address': geo, 'key': 'da4ad28b24dfe117687b257def1bca02'} #浩亮
    parameters = {'address': geo, 'key': '758b19e2580804d3238839e2eabfa59f'} #赟杰
    base = 'https://restapi.amap.com/v3/geocode/geo?parameters'
    loc = 0
    try:
        response = requests.get(base, parameters, timeout=2)
        if response.status_code == 200:
            answer = response.json()
            loc = answer['geocodes'][0]['location']
        else:
            loc='未查到'
    except:
        loc='未查到'
    return loc

In [7]:
# 读取gps
with open('./output/gps信息_20220728','r', encoding='utf8') as f:
    gps_dict_extend = dict()
    for line in f.readlines():
        try:
            a, gps = line.strip().rsplit('|', 1)
            gpslist = gps.split(',')
            gps_dict_extend[a] = gpslist
        except:
            print(line)
            break

In [12]:
# 是否有未查到
i=0
for k, v in gps_dict_extend.items():
    if v==['未查到']:
        i+=1
        print(i,k,v)

1 陕西省 西安市 市辖区 陕西省西安市市辖区新疆霍城县小马扎61团 ['未查到']


In [11]:
# 数量过多，再次查询
for k, v in tqdm(gps_dict_extend.items()):
    if v==['未查到']:
#         k_edit = k
        k_edit = re.sub('^([\\u4e00-\\u9fa5]+\\s){3}', '', k)
        gps_dict_extend[k] = addr2code(k_edit).split(',')

  0%|          | 0/67731 [00:00<?, ?it/s]

In [15]:
addr2code('陕西省西安市市辖区新疆霍城县').split(',')

['108.953939', '34.266611']

In [16]:
gps_dict_extend['陕西省 西安市 市辖区 陕西省西安市市辖区新疆霍城县小马扎61团'] = ['108.953939', '34.266611']

In [17]:
# 打开历史地址
with open('./output/gps_dict.json', 'r') as f:
    gps_dict_base = json.load(f)

In [18]:
# 合并地址
gps_dict = {**gps_dict_base, **gps_dict_extend}

In [33]:
# 保存地址
with open('./output/gps_dict.json', 'w') as f:
    json.dump(gps_dict, f)

## 特征

In [22]:
# 添加gps列
sample_addr['GPS'] = sample_addr['consign_address'].map(lambda x: gps_dict[x])

In [23]:
# 未查得是否有特征
f = lambda x: True if '未查到' in str(x) else False
sample_addr[sample_addr['GPS'].map(f)][['consign_address','GPS']]

,consign_address,GPS


In [24]:
# 是否有空值
f = lambda x: True if len(x)!=2 else False
sample_addr[sample_addr['GPS'].map(f)][['consign_address','GPS']]

,consign_address,GPS


In [25]:
# 添加geohash列
sample_addr['geocode'] = sample_addr['GPS'].map(lambda x: geohash.encode(float(x[1]), float(x[0]), 7))

In [26]:
# 输出宽表
sample_addr[['order_code','GPS','geocode']].to_csv('./output/tmp_addrgps_shl.csv', index=0)